In [69]:
################################################################################
# SETUP
################################################################################

# Convenience and saving flags
ABRIDGED_RUN = True
SAVE_AFTER_TRAINING = True # Save the model when you are done
SAVE_CHECKPOINTS = True # Save the model after ever epoch
REPORT_TRAINING_LOSS_PER_EPOCH = True # Track the training loss each epoch, and write it to a file after training
REPORT_VALIDATION_LOSS_PER_EPOCH = True # Lets us make a nice learning curve after training

# Training hyperparameters
BATCH_SIZE = 96 # Number of samples per batch while training our network
NUM_EPOCHS = 10 # Number of epochs to train our network
LEARNING_RATE = 0.001 # Learning rate for our optimizer

# Directories
CHECKPOINT_DIR = "checkpoints/" # Checkpoints, models, and training data will be saved here
DATA_DIR = "../data/"
AUDIO_DIR = DATA_DIR + "train_audio/"
UNLABELED_DIR = DATA_DIR + "unlabeled_soundscapes/"

# CURRENT PIPELINE:
#  - Split each audio file into 5 second clips
#  - Discard any scrap with duration less than 3.5s. Pad others.
#  - Run Birdcall detection on each clip and change labels appropriately
#  - Loss function is CrossEntropyLoss
#  - Train with freq/time masking, random power, and pink bg noise.
#  - Validate without freq/time masking, random power, or bg noise.
#  - Model output will be a vector of logits. Need to apply sigmoid to get probabilities.


MODEL_NAME = "CE_ALLDATA_PINKBG_WITHDETECT_RANDOMSAMPLE_PSEUDOLABEL"
DETECTION_MODEL = 'BIRDCALL_DETECTION_DCASE_FINAL'

# Preprocessing info
SAMPLE_RATE = 32000 # All our audio uses this sample rate
SAMPLE_LENGTH = 5 # Duration we want to crop our audio to
NUM_SPECIES = 182 # Number of bird species we need to label
MIN_SAMPLE_LENGTH = 3.5 # Only use samples with length >= 3.5 seconds

# Min and max signal to noise ratio
MAX_SNR = 20
MIN_SNR = 10

################################################################################
# IMPORTS
################################################################################

print("Importing modules")

# Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import librosa
import random
import os
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pathlib import Path


# Torch imports
import torch
import torchaudio
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchaudio.transforms import MelSpectrogram, Resample
from IPython.display import Audio
import torch.optim as optim

print("Done")

Importing modules
Done


In [4]:
################################################################################
# LOAD DATA
################################################################################

data = pd.read_csv(DATA_DIR+"full_metadata.csv")
data['filepath'] = AUDIO_DIR + data['filename']

# We only need the filepath, primary label, and duration(?)
data = data[['filepath', 'primary_label', 'duration']]

# Replace string labels by tensors whose entries are dummies
species = ['asbfly', 'ashdro1', 'ashpri1', 'ashwoo2', 'asikoe2', 'asiope1', 'aspfly1', 'aspswi1', 'barfly1', 'barswa', 'bcnher', 'bkcbul1', 'bkrfla1', 'bkskit1', 'bkwsti', 'bladro1', 'blaeag1', 'blakit1', 'blhori1', 'blnmon1', 'blrwar1', 'bncwoo3', 'brakit1', 'brasta1', 'brcful1', 'brfowl1', 'brnhao1', 'brnshr', 'brodro1', 'brwjac1', 'brwowl1', 'btbeat1', 'bwfshr1', 'categr', 'chbeat1', 'cohcuc1', 'comfla1', 'comgre', 'comior1', 'comkin1', 'commoo3', 'commyn', 'compea', 'comros', 'comsan', 'comtai1', 'copbar1', 'crbsun2', 'cregos1', 'crfbar1', 'crseag1', 'dafbab1', 'darter2', 'eaywag1', 'emedov2', 'eucdov', 'eurbla2', 'eurcoo', 'forwag1', 'gargan', 'gloibi', 'goflea1', 'graher1', 'grbeat1', 'grecou1', 'greegr', 'grefla1', 'grehor1', 'grejun2', 'grenig1', 'grewar3', 'grnsan', 'grnwar1', 'grtdro1', 'gryfra', 'grynig2', 'grywag', 'gybpri1', 'gyhcaf1', 'heswoo1', 'hoopoe', 'houcro1', 'houspa', 'inbrob1', 'indpit1', 'indrob1', 'indrol2', 'indtit1', 'ingori1', 'inpher1', 'insbab1', 'insowl1', 'integr', 'isbduc1', 'jerbus2', 'junbab2', 'junmyn1', 'junowl1', 'kenplo1', 'kerlau2', 'labcro1', 'laudov1', 'lblwar1', 'lesyel1', 'lewduc1', 'lirplo', 'litegr', 'litgre1', 'litspi1', 'litswi1', 'lobsun2', 'maghor2', 'malpar1', 'maltro1', 'malwoo1', 'marsan', 'mawthr1', 'moipig1', 'nilfly2', 'niwpig1', 'nutman', 'orihob2', 'oripip1', 'pabflo1', 'paisto1', 'piebus1', 'piekin1', 'placuc3', 'plaflo1', 'plapri1', 'plhpar1', 'pomgrp2', 'purher1', 'pursun3', 'pursun4', 'purswa3', 'putbab1', 'redspu1', 'rerswa1', 'revbul', 'rewbul', 'rewlap1', 'rocpig', 'rorpar', 'rossta2', 'rufbab3', 'ruftre2', 'rufwoo2', 'rutfly6', 'sbeowl1', 'scamin3', 'shikra1', 'smamin1', 'sohmyn1', 'spepic1', 'spodov', 'spoowl1', 'sqtbul1', 'stbkin1', 'sttwoo1', 'thbwar1', 'tibfly3', 'tilwar1', 'vefnut1', 'vehpar1', 'wbbfly1', 'wemhar1', 'whbbul2', 'whbsho3', 'whbtre1', 'whbwag1', 'whbwat1', 'whbwoo2', 'whcbar1', 'whiter2', 'whrmun', 'whtkin2', 'woosan', 'wynlau1', 'yebbab1', 'yebbul3', 'zitcis1']

species_to_index = {species[i]:i for i in range(len(species))}

In [5]:
################################################################################
# GENERATING PSEUDOLABELS ON UNLABELED DATA VIA TRANSFER LEARNING 
################################################################################

# Using ideas from "Transfer Learning with Pseudo Multi-Label Birdcall Classification for DS@GT BirdCLEF 2024" 
import tensorflow as tf
import tensorflow_hub as hub
google_model = hub.load('https://www.kaggle.com/models/google/bird-vocalization-classifier/TensorFlow2/bird-vocalization-classifier/8')
google_labels_path = hub.resolve('https://kaggle.com/models/google/bird-vocalization-classifier/frameworks/tensorFlow2/variations/bird-vocalization-classifier/versions/8') + "/assets/label.csv"

In [46]:
google_labels = pd.read_csv(google_labels_path)
labels_2021 = list(google_labels['ebird2021'])

In [71]:
mask = np.zeros(len(labels_2021))

for s in species:
    if s in labels_2021:
        index = google_labels.index[google_labels['ebird2021'] == s].tolist()[0]
        mask[index] = 1

mask = np.array(mask, dtype=bool)

In [81]:
unlabeled_soundscapes = sorted(os.listdir(UNLABELED_DIR))
step_size = SAMPLE_RATE * 5
count = 0 

if ABRIDGED_RUN == True:
    unlabeled_soundscapes = random.sample(unlabeled_soundscapes, 10)
else:
    unlabeled_soundscapes = random.sample(unlabeled_soundscapes, 800)
    
# for path in tqdm(unlabeled_soundscapes):
#     num_segments = 240 // 5
#     waveform, sr = librosa.load(UNLABELED_DIR + path, sr=SAMPLE_RATE)
#     waveform = waveform.astype(np.float32)
#     dur = librosa.get_duration(y=waveform, sr=sr)
    
    
#     if len(waveform) < 240 * SAMPLE_RATE:
#         continue
    
    
#     for i in tqdm(range(num_segments)):
#         x = waveform[(step_size*i):(step_size*(i+1))]
#         logits = google_model.infer_tf(x[np.newaxis, :])
#         logits = logits['label'].numpy()[0]
#         logits = logits[mask]
    
#         pred_label = google_labels.iloc[[np.argmax(logits)]]['ebird2021'].item() 
        
#         if pred_label in species: 
#             print("Adding new row to metadata")
#             count += 1
#             new_row = {
#             'filepath': UNLABELED_DIR + path,
#             'primary_label': pred_label,
#             'duration': dur
#             }
#             data = pd.concat([data, pd.DataFrame([new_row])])
#             break 
    

# print(f"Added {count} datapoints from unlabeled soundscape")

for path in tqdm(unlabeled_soundscapes):
    waveform, sr = librosa.load(UNLABELED_DIR + path, sr=SAMPLE_RATE)
    waveform = waveform.astype(np.float32)
    dur = librosa.get_duration(y=waveform, sr=sr)
    
    
    if len(waveform) < 240 * SAMPLE_RATE:
        continue
    
    
    for i in range(4):
        start = np.random.randint(waveform.shape[0]-SAMPLE_RATE*SAMPLE_LENGTH+1)

        x = waveform[start:start + SAMPLE_RATE*SAMPLE_LENGTH]
        logits = google_model.infer_tf(x[np.newaxis, :])
        logits = logits['label'].numpy()[0]
        logits = logits[mask]
        pred_label = google_labels.iloc[[np.argmax(logits)]]['ebird2021'].item() 
        
        if pred_label in species: 
            print("Adding new row to metadata")
            count += 1
            new_row = {
            'filepath': UNLABELED_DIR + path,
            'primary_label': pred_label,
            'duration': dur
            }

        data = pd.concat([data, pd.DataFrame([new_row])])
    


print(f"Added {count} datapoints from unlabeled soundscape")



  0%|          | 0/10 [00:00<?, ?it/s]

[-16.182888 -10.089521 -17.00353  ... -18.126009 -15.91088  -13.197889]
[-12.140304    -9.536605   -15.159098   -17.414268    -9.26193
 -18.59745    -11.652218   -15.857957   -10.374762   -11.004333
  -8.6138115  -12.852977   -13.015774   -10.015996   -10.748857
 -12.932539   -10.830873    -8.7248955   -8.197229   -11.206668
 -14.371604   -17.546017   -16.054077    -7.0142694  -13.986792
 -11.325673   -14.691146    -8.70832    -17.267355   -10.21945
 -16.227596   -12.683728   -12.275227   -12.37685    -14.677467
 -12.145354    -8.851334    -9.397553    -8.705748    -5.918978
 -10.918662   -10.351061   -10.846949    -7.2676044   -8.825595
 -10.443416    -9.847268   -14.213484   -13.613717    -9.316313
  -9.201786   -14.071221    -8.616915   -10.9564905   -9.4638
 -11.377034    -7.354533   -12.737599   -11.398599   -15.457252
 -10.872189    -9.095661   -12.981686    -8.135442   -11.680374
  -9.372511   -10.965663    -5.415431   -15.417003    -8.822309
  -8.913292   -11.486623   -10.76679

[-15.348285 -10.758653 -15.321203 ... -16.893723 -15.967014 -13.763664]
[-11.362393   -9.558344  -13.970017  -17.353964   -8.313494  -16.743874
 -10.6454115 -13.265311   -7.6191697 -11.808588   -7.161933   -8.44035
 -13.819398  -10.384457  -11.179996  -12.011367   -7.5895534  -8.927334
  -8.056838  -12.481757  -13.965495  -17.31228   -17.206177   -6.1871696
 -13.101134  -11.941836  -14.138417   -7.4883566 -17.05321   -11.049992
 -15.497989  -11.447145  -13.559957  -11.561776  -12.564599  -11.320187
  -8.507873   -9.7153425  -9.507276   -5.5713296  -9.192698   -9.944891
  -9.856054   -8.381234   -9.93821    -8.650188   -9.108253  -13.561369
 -11.786072   -7.9885683  -8.0727825 -12.960936   -8.465901   -6.479889
  -9.392046   -7.7102714  -6.453208  -12.85343   -10.616078  -15.317446
 -11.236885   -9.173137  -13.192703   -8.760444  -10.575289  -10.913146
  -8.920128   -2.4304976 -14.417489   -9.002702   -9.223293  -11.064366
  -9.0742    -15.117284  -15.0643425  -8.508969  -13.655769   -9

[-17.05757  -11.525559 -18.700148 ... -18.578142 -19.21122  -14.377315]
[-11.974232  -10.812713  -16.04171   -19.453636  -10.123717  -19.64402
 -11.742747  -16.066809   -8.526633  -10.957246   -7.7391386  -6.7066426
 -16.15545   -11.080449  -11.612846  -14.9026575  -9.718093  -10.006782
 -10.19087   -12.087269  -15.326747  -19.598902  -17.403471   -7.265856
 -14.3763685 -14.016483  -15.930182   -9.311806  -18.862942  -11.165615
 -18.251123  -14.215908  -14.174619  -11.987307  -13.604885  -15.515032
  -7.6839266  -9.600037   -8.817249   -6.549249  -11.258695  -12.264466
  -9.786207   -7.7936163 -12.096609  -11.78238   -11.25362   -17.855026
 -14.106731   -8.931691   -8.649327  -17.91407    -7.6642222 -11.218516
  -9.948968  -10.3487835  -6.139553  -13.549328  -12.132354  -17.398655
 -11.782656  -10.150867  -14.010688  -10.449619  -12.001618  -10.964161
 -11.048334   -3.9125264 -13.340461   -9.61997    -7.8241987 -13.28901
  -9.63088   -14.874231  -17.196035   -8.878504  -14.826193  -11.

[-15.601288 -11.209043 -16.894306 ... -16.638704 -15.708735  -9.711019]
[ -8.152239  -10.693705  -13.965099  -17.63632    -9.755977  -17.355137
  -9.594298  -13.648219   -8.155355  -11.352509   -7.8667264  -5.5421863
 -14.826646  -11.285253  -12.600666  -11.107434  -10.0405245  -8.117164
  -8.741984  -11.68797   -12.295974  -15.929054  -16.79174    -5.660163
 -12.898657  -11.072651  -14.516014   -7.542643  -18.389988  -11.286757
 -14.045475  -12.615016  -13.387349   -9.69118   -12.234995  -12.674728
  -7.6159906  -9.209259   -7.6033382  -6.3603477  -9.081952  -10.371889
  -9.757185   -7.53646    -7.8187833  -8.12741    -8.612406  -14.044363
 -11.807469   -7.8099437  -8.723592  -14.095173   -8.496742  -10.041669
  -9.534459   -8.390752   -7.382246  -12.458094  -11.494764  -14.904117
 -10.636522   -9.112879  -12.296346   -9.310435  -12.431448  -10.2880335
  -9.415657   -2.4976995 -14.232031   -7.1368265  -8.43322   -10.632404
  -8.2008095 -13.728159  -16.199354   -8.949969  -12.761793  -

[-16.379719 -10.453501 -15.66096  ... -19.297249 -19.318588 -14.171504]
[-11.391192  -10.560214  -14.719167  -19.053596  -10.519624  -16.571054
 -11.51387   -14.53553    -7.7954655 -11.354489   -8.950671   -8.873963
 -11.503669  -10.705521  -10.126854  -13.237711   -9.478981   -9.577209
  -9.298851  -12.060806  -15.11545   -18.055782  -17.963432   -5.9903846
 -16.930641  -12.865107  -15.49391    -8.476657  -18.197115  -13.983986
 -15.544422  -11.410958  -13.065242  -11.107218  -14.364076  -12.501109
  -8.183333   -8.244851   -9.453257   -5.990736  -10.362259  -10.70768
 -11.61987    -7.979307   -9.482979   -8.162142   -9.449031  -15.2501
 -13.017265   -7.503604   -9.1685095 -14.609624   -8.970085  -10.454048
  -9.631452  -10.602648   -6.168769  -15.116363  -10.914024  -14.830975
 -12.637926  -10.077197  -12.014001  -10.274631  -12.44825   -11.385899
 -10.375581   -5.479348  -14.882657   -9.007939   -8.944242  -12.67725
  -9.823525  -15.331909  -16.11495    -9.499203  -13.305759  -10.75

[-15.635172  -9.196008 -16.463526 ... -18.175106 -14.552186 -11.667057]
[-11.329766   -8.720218  -13.112525  -15.535604   -7.7091575 -17.288721
  -9.359634  -14.304786   -7.4720345 -10.046458   -7.10174    -8.899191
 -12.134792  -10.086561   -9.993014  -14.148719   -9.949081   -7.934282
  -8.876161  -10.4953575 -11.450548  -16.32067   -15.60618    -5.8343983
 -12.639773  -10.081034  -13.318407   -7.3055143 -15.170306   -9.087476
 -14.80859   -10.428169  -13.279226  -11.161552  -12.445295  -10.7390785
  -7.8406296  -9.337217   -9.090274   -4.8488126  -8.368956   -9.919594
  -7.8075824  -6.8842998  -7.244624   -9.112443   -8.132311  -13.96734
 -12.499348   -7.126862   -7.8725095 -13.593781   -8.010386  -10.793465
  -9.283366  -11.591519   -6.1594763 -11.361806  -10.557654  -15.110458
  -8.048146   -9.612197  -10.664937   -7.817873  -11.038181  -10.455446
 -11.027363   -2.9751415 -14.868645   -8.068899   -7.946876   -9.771543
  -8.459249  -14.378116  -14.358719   -7.3341484 -11.945552   -

  0%|          | 0/10 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
data['index_label'] = data['primary_label'].apply(lambda x: species_to_index[x])
data['tensor_label'] = pd.Series(pd.get_dummies(data['primary_label']).astype(int).values.tolist()).apply(lambda x: torch.Tensor(x))

In [105]:
# Remove overly short clips
data = data[data['duration'] >= MIN_SAMPLE_LENGTH]

# Use 100 rows of data for quick runs to test functionalities
if ABRIDGED_RUN == True:
    data = data.sample(100)

# Progress bars for loading data to memory
tqdm.pandas()

# print("Loading nocall background noise snippets into memory")
# nocalls = pd.read_csv(f"{DATA_DIR}nocall_snippets/filenames.csv")
# def load_nocall(filepath):
#    signal, _ = torchaudio.load(filepath)
    # Reduce to one channel
#    signal = torch.mean(signal,dim = 0).unsqueeze(0)
#    return signal
# nocalls['signal'] = nocalls['filename'].progress_apply(lambda x: load_nocall(f"{DATA_DIR}nocall_snippets/{x}"))
# print("Done")

print("Loading training audio signals into memory")
# Loads signal to memory, pads to 5 seconds, cuts to 60 seconds
def filepath_to_signal(filepath):
    signal, _ = torchaudio.load(filepath)
    return signal
data['signal'] = data['filepath'].progress_apply(filepath_to_signal)
print("Done")

# Train test split, stratified by species
stratify = data['primary_label']
if ABRIDGED_RUN == True:
    stratify = None
data_train, data_validation = train_test_split(data, test_size = 0.2, stratify=stratify)

KeyError: 'afcdov1'

In [27]:
################################################################################
# PREPROCESSING FUNCTIONS
################################################################################

# Transforms audio signal to a spectrogram
spectrogram_transform = torchaudio.transforms.Spectrogram(
        n_fft=2048,
        win_length=2048,
        hop_length=512,
        power=2
    )

# Converts ordinary spectrogram to Mel scale
mel_spectrogram_transform = torchaudio.transforms.MelScale(
    n_mels=128,
    sample_rate=SAMPLE_RATE,
    f_min=40,
    f_max=16000,
    n_stft=1025  # the number of frequency bins in the spectrogram
)

# Scales decibels to reasonable level (apply to a spectrogram or Mel spectrogram)
db_scaler = torchaudio.transforms.AmplitudeToDB(stype="power", top_db=80)

# Resizes spectrograms into square images
resize = transforms.Resize((224, 224), antialias = None)

# Applies a frequency mask to a spectrogram
def freq_mask(spec, F=30):
    num_mel_channels = spec.shape[1]
    f = random.randrange(0, F)
    f_zero = random.randrange(0, num_mel_channels - f)
    spec[:, f_zero:f_zero+f, :] = 0
    return spec

# Applies a time mask to a spectrogram
def time_mask(spec, T=40):
    spec_len = spec.shape[2]
    t = random.randrange(0, T)
    t_zero = random.randrange(0, spec_len - t)
    spec[:, :, t_zero:t_zero+t] = 0
    return spec

# Taken from https://www.earthinversion.com/datascience/pink_noise_vs_white_noise/
def generate_pink_noise(samples):
    b = [0.049922035, -0.095993537, 0.050612699, -0.004408786]
    a = [1, -2.494956002, 2.017265875, -0.522189400]

    pink_noise = np.random.randn(samples)
    pink_noise = np.convolve(pink_noise, b)
    pink_noise = np.convolve(pink_noise, a, mode='valid')
    return torch.Tensor(pink_noise).unsqueeze(0)

# Signal is a torch tensor of shape [1, _]
# snr is signal to noise ratio in dB, a float
def add_pink_noise(signal, snr):
    pink = generate_pink_noise(signal.shape[1])
    return torchaudio.functional.add_noise(signal, pink, torch.Tensor([snr]))

# def add_bg_noise(signal, snr):
#    bg_noise = nocalls['signal'].sample(1).iloc[0]
#    return torchaudio.functional.add_noise(signal, bg_noise, torch.Tensor([snr]))


In [ ]:
################################################################################
# IMPORT BIRDCALL DETECTION MODEL
################################################################################


class BirdCallDetector(nn.Module):
    ''' Full architecture from https://github.com/musikalkemist/pytorchforaudio/blob/main/10%20Predictions%20with%20sound%20classifier/cnn.py'''
    def __init__(self):
        super(BirdCallDetector, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(10368, 2)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.flatten(x)
        logits = self.linear(x)
        predictions = self.softmax(logits)
        return predictions

# Set device we'll train on    
device = None
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device}")

detection_model = BirdCallDetector().to(device)
detection_model.load_state_dict(torch.load(CHECKPOINT_DIR + DETECTION_MODEL + '/final.pt', map_location=torch.device(device)))
detection_model.eval()

################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################



In [ ]:
################################################################################
# DATASET CLASS
################################################################################

# Slice a sequence into segments
def slices(seq, window_size = SAMPLE_RATE*SAMPLE_LENGTH, stride = None, align_left = True):
    # If one window is larger than the sequence, just return the scraps or nothing
    if window_size > seq.shape[0]:
        if seq.shape[0] < SAMPLE_RATE*MIN_SAMPLE_LENGTH:
            return []
        pad_length = SAMPLE_RATE*SAMPLE_LENGTH - seq.shape[0]
        if align_left == False:
            seq = torch.flip(seq, [0])
            seq = torch.nn.functional.pad(seq, (0, pad_length))
            return [torch.flip(seq, [0])]
        return [torch.nn.functional.pad(seq, (0, pad_length))]
        
    # If stride is None, it defaults to window_size
    if stride == None:
        stride = window_size

    index_slices = []
    left_pointer = 0
    while left_pointer + window_size <= seq.shape[0]:
        index_slices += [[left_pointer, left_pointer + window_size]]
        left_pointer += stride

    if align_left == False:
        offset = seq.shape[0]-(left_pointer-stride)-window_size
        index_slices = [[a+offset, b+offset] for [a,b] in index_slices]

    result = [seq[a:b] for [a,b] in index_slices]

    if align_left == True and left_pointer < seq.shape[0]:
        scrap = seq[left_pointer : seq.shape[0]]
        pad_length = window_size - (seq.shape[0]-left_pointer)
        scrap = torch.nn.functional.pad(scrap, (0, pad_length))
        result.append(scrap)

    elif align_left == False and index_slices[0][1] > stride:
        scrap = seq[0: index_slices[0][1]-stride]
        scrap = torch.flip(scrap, [0])
        pad_length = window_size - (index_slices[0][1]-stride)
        scrap = torch.nn.functional.pad(scrap, (0, pad_length))
        result = [torch.flip(scrap, [0])] + result
    
    return result



In [ ]:
# Note: signals and labels should be ordinary lists
# Training boolean is used to decide whether to apply masks
# Config should have the format of a dictionary
class BirdDataset(Dataset):
    def __init__(self, signals, labels, 
                 training = True,
                 use_mel = True,
                 use_time_mask = True, 
                 use_freq_mask = True, 
                 use_pink_noise = True):
        super().__init__()
        self.training = training
        self.use_mel = use_mel
        self.use_time_mask = use_time_mask
        self.use_freq_mask = use_freq_mask
        self.use_pink_noise = use_pink_noise
        print(f'Preprocessing {"training" if training else "validation"} data\n')
        self.processed_clips, self.labels = self.process(signals, labels)

    def process(self, signals, labels):
        processed_clips = []
        processed_labels = []
        for i, signal in enumerate(tqdm(signals, total = len(signals), leave = False)):
            # Cut signal into 5 second chunks and process each clip separately
            clips = slices(signal.squeeze())
            clip_labels = []

            for clip in clips:
                x = clip.unsqueeze(0)
                x = spectrogram_transform(x)
                x = mel_spectrogram_transform(x)
                x = db_scaler(x)
                x = resize(x)
                x = x.to(device)
                prob_birdcall = detection_model(x.unsqueeze(0))[0][1].item()
                if prob_birdcall >= 0.5:
                    clip_labels += [labels[i]]
                else:
                    clip_labels += [torch.zeros(len(species))]
            if len(clips) > 0:
                processed_labels += [clip_labels]
                processed_clips += [[x.unsqueeze(0) for x in clips]]

        return processed_clips, processed_labels

    def __len__(self):
        return len(self.processed_clips)

    def __getitem__(self, index):
        segments = self.processed_clips[index]
        segment_labels = self.labels[index]

        segment_index = np.random.randint(low = 0, high = len(segments))

        x = segments[segment_index]
        label = segment_labels[segment_index]

        # Add pink noise
        if self.use_pink_noise and self.training:
            x = add_pink_noise(x, np.random.uniform(low = MIN_SNR, high = MAX_SNR))

        # Process
        x = spectrogram_transform(x)
        if self.use_mel:
            x = mel_spectrogram_transform(x)
        if self.training:
            exponent = np.random.uniform(low = 0.5, high = 3)
            x = torch.pow(x, exponent)
        x = db_scaler(x)
        if self.use_time_mask and self.training:
            x = time_mask(x)
        if self.use_freq_mask and self.training:
            x = freq_mask(x)
        x = resize(x)
        return x, label


In [ ]:
################################################################################
# ARCHITECTURE
################################################################################


class BirdClassifier(nn.Module):
    ''' Full architecture from https://github.com/musikalkemist/pytorchforaudio/blob/main/10%20Predictions%20with%20sound%20classifier/cnn.py'''
    def __init__(self, num_classes):
        super(BirdClassifier, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.Dropout()
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.Dropout()
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(46208, num_classes)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x


In [ ]:
################################################################################
# WEIGHTED RANDOM SAMPLER SETUP
################################################################################

#label_counts = data_train['primary_label'].value_counts()

#total_durs = data_train.groupby(by='primary_label')['duration'].sum()
#total_durs = data_train['primary_label'].apply(lambda x: total_durs[x])

#weights = data_train['primary_label'].apply(lambda x: 1/label_counts.loc[x])*(data_train['duration']/total_durs)
#weights = data_train['duration']/total_durs
#weighted_sampler = torch.utils.data.WeightedRandomSampler(weights.to_list(), len(weights))





################################################################################
# TRAINING SETUP
################################################################################

# Create a saving directory if needed
output_dir = Path(f'{CHECKPOINT_DIR}{MODEL_NAME}')
output_dir.mkdir(parents=True, exist_ok=True)
output_dir = f'{CHECKPOINT_DIR}{MODEL_NAME}'

# Instantiate our training dataset
train_dataset = BirdDataset(signals = data_train['signal'].to_list(), 
                            labels = data_train['tensor_label'].to_list(),
                            training = True,
                            use_pink_noise = True)
train_dataloader = DataLoader(train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

# Instantiate our validation dataset
validation_dataset =  BirdDataset(signals = data_validation['signal'].to_list(), 
                                  labels = data_validation['tensor_label'].to_list(),
                                  training = False)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Instantiate our model
model = BirdClassifier(NUM_SPECIES).to(device)

# Set our loss function and optimizer
pos_weight = torch.ones([len(species)]).to(device)
criterion = torch.nn.CrossEntropyLoss(pos_weight=pos_weight).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

# Training loop
print(f"Training on {len(train_dataset)} samples with {BATCH_SIZE} samples per batch.")
if REPORT_VALIDATION_LOSS_PER_EPOCH == True:
    print(f"Validating on {len(validation_dataset)} samples at the end of each epoch.")

training_losses = [None]*NUM_EPOCHS
validation_losses = [None]*NUM_EPOCHS

torch.enable_grad() # Turn on the gradient



In [ ]:
################################################################################
# SAVE AND REPORT train test split
################################################################################

# Save train test split
data_train[['primary_label', 'filepath']].to_csv(f"{output_dir}/data_train.csv", index = False)
data_validation[['primary_label', 'filepath']].to_csv(f"{output_dir}/data_validation.csv", index = False)

In [ ]:
################################################################################
# TRAINING LOOP
################################################################################

for epoch_num, epoch in enumerate(tqdm(range(NUM_EPOCHS), leave = False)):

    running_loss = 0.0
    
    for i, data in enumerate(tqdm(train_dataloader, leave = False)):
        
        # Get batch of inputs and true labels, push to device
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass on batch of inputs
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Update weights
        optimizer.step()

        # Update loss
        running_loss += loss.item()

    # Save checkpoint
    if SAVE_CHECKPOINTS == True:
        torch.save(model.state_dict(), f"{CHECKPOINT_DIR}{MODEL_NAME}/checkpoint_{epoch_num+1}.pt")

    # Compute training loss
    if REPORT_TRAINING_LOSS_PER_EPOCH == True:    
        training_losses[epoch_num] = running_loss/len(train_dataloader)
        
    # Compute validation loss
    if REPORT_VALIDATION_LOSS_PER_EPOCH == True:
        validation_loss = 0.0
        model.eval()
        torch.no_grad()
        for validation_data in validation_dataloader:
            inputs, labels = validation_data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            validation_loss += criterion(outputs, labels).item()
        validation_losses[epoch_num] = validation_loss/len(validation_dataloader)
        model.train()
        torch.enable_grad()

    # Save losses
    losses = pd.DataFrame({"training_losses":training_losses, "validation_losses":validation_losses})
    cols = []
    if REPORT_TRAINING_LOSS_PER_EPOCH == True:
        cols += ["training_losses"]
    if REPORT_VALIDATION_LOSS_PER_EPOCH == True:
        cols += ["validation_losses"]
    if len(cols) > 0:
        losses[cols].to_csv(f'{output_dir}/losses.csv', index = False)

print('Finished Training')


In [ ]:
################################################################################
# SAVE AND REPORT 
################################################################################

# Save model
if SAVE_AFTER_TRAINING == True:
    torch.save(model.state_dict(), f'{output_dir}/final.pt')

################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################